In [ ]:
!pip install faiss-cpu pypdf2

In [1]:
import faiss
from langchain.embeddings import OllamaEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [2]:
# pdf_file_path = 'C:/Users/ribhattacharya/Desktop/TPRM-Accelerator/assets/data/Security Evidence Docs/SOC 2/GoogleCloud/Audit-Reports-1720774833381-81ba2e/GCP-[FALL-2023] GCP SOC 2..pdf'
# with fitz.open(pdf_file_path) as doc:
#         page_contents = [page.get_text() for page in doc]
pdf_loader = PyPDFLoader("C:/Users/ansutton/Desktop/TPRM/TPRM-Accelerator/assets/data/Security Evidence Docs/SOC 2/GoogleCloud/Audit-Reports-1720774833381-81ba2e/GCP-[FALL-2023] GCP SOC 2..pdf")
raw_documents= pdf_loader.load()
text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=100,
        length_function=len,
        is_separator_regex=False,
    )
documents = text_splitter.split_documents(raw_documents)

In [3]:
embedding_model = OllamaEmbeddings(model='nomic-embed-text',show_progress=True)

In [4]:
batch_size = 50
texts=[doc.page_content for doc in documents]
#document_texts = [doc.page_content for doc in documents]
documents_embeddings = []

In [5]:
for i in range(0, len(texts),batch_size):
    batch_texts = texts[i:i+batch_size]
    batch_embeddings = embedding_model.embed_documents(batch_texts)
    documents_embeddings.extend(batch_embeddings)

OllamaEmbeddings: 100%|██████████| 30/30 [00:14<00:00,  2.11it/s]


In [6]:
import numpy as np

In [7]:
#create FAISS index and add the embeddings
document_embeddings_np = np.array(documents_embeddings)
dimension = document_embeddings_np.shape[1]
faiss_index = faiss.IndexFlatL2(dimension)
faiss_index.add(document_embeddings_np)

In [8]:
from langchain.docstore import InMemoryDocstore

In [9]:
docstore = InMemoryDocstore(dict(enumerate(documents)))
index_to_docstore_id = {i: i for i in range(len(documents))}

In [10]:
vector_store = FAISS(embedding_model, faiss_index, docstore, index_to_docstore_id)

In [11]:
from langchain_community.chat_models import ChatOllama
from langchain.chains import RetrievalQA

In [12]:
local_model = "llama3.2" 
llm = ChatOllama(model=local_model)

In [13]:
qa_chain = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type="stuff",
    retriever = vector_store.as_retriever(search_kwargs={"k":3}),
    return_source_documents = True
)

In [14]:
def get_answer(query):
    result = qa_chain({"query": query})
    response = result['result']

    source_documents = result['source_documents']
    pages = set()
    citations = []
    for doc in source_documents:
        pages.add(doc.metadata['page'])
        citations.append((doc.metadata['page'], doc.page_content))

    return {
        "response": response,
        "pages": list(pages),
        "citations": citations
    }

In [15]:
query = "What access control procedures are in place?"

In [16]:
structured_answer = get_answer(query)

c:\Users\ansutton\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
OllamaEmbeddings:   0%|          | 0/1 [00:00<?, ?it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  8.28it/s]


In [17]:
print(structured_answer["response"])

Based on the provided context, I can infer that there are several access control procedures in place. Here are some of the specific procedures mentioned:

1. Registration and authorization of new internal and external users before granting them system access.
2. Removal of user system credentials when user access is no longer authorized.
3. Authorization, modification, or removal of access to data, software, functions, and other protected information assets based on roles, responsibilities, or the system design.
4. Implementation of logical access security software, infrastructure, and architectures over protected information assets.

Additionally, customers are responsible for reviewing users' access rights periodically, consistent with their organizational policies, to mitigate the risk of inappropriate access.

Note that Common Criteria 6.1 specifically mentions implementing logical access security software, infrastructure, and architectures, which suggests a structured approach to 

In [18]:
print(structured_answer["pages"])

[58, 60, 63]


In [19]:
print(structured_answer["citations"])

[(60, "assesses those parties’ compliance \non a periodic and as -needed basis \nand takes corrective action, if \nnecessary.  \nCustomer s are responsible for provisioning, \nmaintaining, monitoring and disabling end users’ \naccess in accordance with their internal access \nmanagement policies.  Common Criteria 6.1: The entity \nimplements logical access security \nsoftware, infrastructure, and \narchi tectures over protected \ninformation assets to protect them \nfrom security events to meet the \nentity's objectives.  \nCommon Criteria 6.2: Prior to \nissuing system credentials and \ngranting system access, the entity \nregisters and authorizes new internal \nand external users whose access is \nadministered by the entity. For those \nusers whose access is administered \nby the entity,  user system rishav.bhattacharya99@gmail.comGoogle Confidental Information"), (58, 'procedures.  Common Criteria 4.1: The entity \nselects, develops, and performs \nongoing and/or separate evaluation

In [20]:
questions = [
"When was the access control policy last reviewed?",
"What is the password management policy in place?",
"What procedures are followed for decommissioning of equipments?",
"What steps are taken to ensure removal of client information prior to decommissioning of equipments?",
"When were information security policies and procedures updated?",
"How are emergency changes performed?",
"What cryptographic controls are in place to encrypt the data at rest and in motion?",
"When was the incident management policy last reviewed?",
"When was the incident response procedure tested?",
"How are incidents classified?",
"What procedures are in place to perform root cause analysis for an incident?",
"What physical perimeter controls are in place?",
"What procedures are in place to deploy patches throughout the IT infrastructure?",
"How is a back up of the data performed?",
"What backgound checks are performed during hiring of an employee?",
"What security incident response procedures are in place?"
]

In [21]:
all_answers = []

In [22]:
for question in questions:
    structured_answer = get_answer(question)
    all_answers.append(structured_answer)

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 16.10it/s]


In [23]:
for answer in all_answers:
    print(answer["response"])
    print(answer["pages"])
    print(answer["citations"])

I don't know when the access control policy was last reviewed based on the information provided. The text mentions that security and privacy policies are reviewed at least annually, but it does not specifically mention the review of an "access control policy". However, it does mention that user access to high-security areas in data centers was reviewed on a quarterly basis, which suggests that some level of access control monitoring is in place.
[144, 159, 119]
[(119, 'security and privacy policies are reviewed at least \nannually, and supporting standards, guidelines, \nand FAQs are created and updated as needed.  No deviations noted.  \nInspected internal documentation and determined \nthat security and privacy policies, supporting \nstandards, guidelines and FAQS were in place.  No deviations noted.  \nInspected internal documentation and determined \nsecurity and privacy policies were reviewed at \nleast annually and authorized before they were \nimplemented.  No deviations noted. 